## Overview

The EMISOpen consultation writeback is essentially a combination of several HL7 v2 Events.

### ADT_A04 - Register a Patient (patient encounter)

| HL7 v2.4 Segment                                                     | HL7 FHIR Resource                                                                                                    | EMISOpen                                        |
|----------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------|-------------------------------------------------|
| [PID](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PID)   | [Patient.identifier](https://virtually-healthcare.github.io/R4/StructureDefinition-Patient.html)                     | RegistrationType (extends IdentType)            |
| [PV1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PV1})  | [Encounter](https://virtually-healthcare.github.io/R4/StructureDefinition-Encounter.html)                            | ConsultationType                                |
| - [OBX](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/OBX) | [Observation](https://virtually-healthcare.github.io/R4/StructureDefinition-Observation.html)                        | EventType (both coded and uncoded)              |
| OBX (type = ED)                                                      | [DocumentReference](https://virtually-healthcare.github.io/R4/StructureDefinition-DocumentReference.html) and Binary | Attachment                                      |
| OBX (dependent on code)                                              | [Task](https://virtually-healthcare.github.io/R4/StructureDefinition-Task.html)                                      | DiaryType                                       |
| OBX (dependent on code)                                              | [Immunization](https://virtually-healthcare.github.io/R4/StructureDefinition-Immunization.html)                      | EventType                                       |
| - [AL1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/AL1) | [AllergyIntolerance](https://virtually-healthcare.github.io/R4/StructureDefinition-AllergyIntolerance.html)                                                                                              | AllergyType                                     |
| - [DG1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/DG1) | [Condition](https://virtually-healthcare.github.io/R4/StructureDefinition-Condition.html)                            | EventType (with problem elements)               |
| - [PR1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PR1) | [Procedure](https://virtually-healthcare.github.io/R4/StructureDefinition-Procedure.html)                            | EventType (can be inferred from SNOMED CT code) |

### RDE_O11 - Pharmacy/treatment encoded order

| HL7 v2.4 Segment                                                    | HL7 FHIR Resource                                                                                         | EMISOpen                             |
|---------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|--------------------------------------|
| [PID](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PID)  | [Patient.identifier](https://virtually-healthcare.github.io/R4/StructureDefinition-Patient.html)          | RegistrationType (extends IdentType) |
| [PV1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PV1}) | [Encounter](https://virtually-healthcare.github.io/R4/StructureDefinition-Encounter.html)                 | ConsultationType                     |
| [RXE](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/RXE)  | [MedicationRequest](https://virtually-healthcare.github.io/R4/StructureDefinition-MedicationRequest.html) | MedicationType                       |



### Miscellaneous HL7 v2 Segments

| HL7 v2.4 Segment                                                   | HL7 FHIR Resource                                                                                       | EMISOpen          |
|--------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|-------------------|
| [ORC](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/ORC) | [ServiceRequest](https://virtually-healthcare.github.io/R4/StructureDefinition-ServiceRequest.html)     | ReferralType      |
| [OBR](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/ORC) | [DiagnosticReport](https://virtually-healthcare.github.io/R4/StructureDefinition-DiagnosticReport.html) | InvestigationType |

This is not a 100% accurate map, but we can see HL7 v2, FHIR and EMIS are related. Similar maps can be created to other GP systems, and they are similar. One of the main reasons for this is UK GP systems were based on early HL7, so they also share similar models with many secondary care systems and laboratory systems.

We will use FHIR as:

1. We internally use FHIR as our [Canonical Data Model](https://www.enterpriseintegrationpatterns.com/patterns/messaging/CanonicalDataModel.html). The link provides a brief description of the decision behind this (the book has a more comprehensive description).
2. The tooling to support FHIR models is quite extensive.

## HL7 v2 vrs FHIR

We don't view v2 and FHIR as competing. HL7 v2 is primarily a workflow interaction standard that supports asynchronous input/output. FHIR, on the other hand, is more focused on REST API interactions for querying data, which is something HL7 v2 doesn't support (well). As mentioned previously, we also use FHIR as the master data model as it can be converted to other standards such as HL7 v2 and XDS quite easily.





## Consultation Writeback

The writeback is a complex workflow. The diagram below is the intitial design for the updated writeback which includes the ability to send to GP Practices which don't use virtually via GP Connect Send Document.

![Alt text](../images/Writeback.drawio.png)

### Application - Clinical Process

A practitioner:

- GP - creates a clinical management plan
- Pharmacist - creates a consultation note

Both are stored in the Clinical Data Repository (FHIR Repository) and then a FHIR Task is created with status `requested`, requesting the consultation to be sent to the patient's GP.

### Workflow

Tasks are polled from the FHIR Repository. For each Task:

- **Extract Encounter**. Read consultation from the FHIR Repository and update FHIR Task to `accepted`
- **Validate** TODO Use FHIR Validation to ensure it is valid. If not reject (and set Task.status to rejected)
- **Get PDS Record** TODO Lookup Patients GP from PDS (**NEW**)
- **Lookup Endpoint**  TODO Get Endpoint details to check to see if this is a local delivery.
- If **EMIS (local delivery)**
  - TODO Check for duplicates
  - Convert to EMISOpen
  - Send consultation to EMIS via IM1 Transaction
- elseIf **TPP (local delivery)**
  - Convert to TPP
  - Send consultation to TPP via IM1 Transaction
- Else
  - Convert to a FHIR Document (**NEW**)
  - Convert FHIR Document to PDF (**NEW**)
  - Send consultation to GP Practice via MESH (workflow: GP Connect)
- Update FHIR Task to `completed`.

Note: For conformance reasons, a pharmacist should not use IM1 Transaction for

Exceptions result in the FHIR Task status being changed to `cancelled`, validation errors have `failed` status



### Example set up

This gets a previously completed task and resets it

1. This is `GET Task\8b12d352-b2dc-4647-a544-ea2ed30011e1`
2. update `status` to `accepted`
3. Update the Task via `POST Task\8b12d352-b2dc-4647-a544-ea2ed30011e1`

In [212]:
import requests
import json
import time

headersCDR = { "Accept": "application/fhir+json"}

cdrFHIRUrl = "http://localhost:8180/CDR/FHIR/R4"

tasks = []
# first is already present on emis, 2nd should fail
for taskid in ['8b12d352-b2dc-4647-a544-ea2ed30011e1','17eb465f-3c31-48df-b6be-37600e15712c']:
    response = requests.get(cdrFHIRUrl + '/Task/' + taskid,headers=headersCDR)
    if response.status_code == 200:
        taskJSON = json.loads(response.text)
        taskJSON['status'] = 'accepted'
        headersCDR = {"Content-Type": "application/fhir+json", "Accept": "application/fhir+json"}
        response = requests.put(cdrFHIRUrl + '/Task/'+taskid,json.dumps(taskJSON),headers=headersCDR)
        print(response.text)


{
  "resourceType": "Task",
  "id": "8b12d352-b2dc-4647-a544-ea2ed30011e1",
  "meta": {
    "versionId": "397",
    "lastUpdated": "2025-06-26T12:25:44.066Z"
  },
  "identifier": [ {
    "system": "https://fhir.virtually.healthcare/Id/Task",
    "value": "537fd315-8a60-4c02-a96e-473bd04a138d"
  } ],
  "status": "accepted",
  "intent": "order",
  "priority": "routine",
  "code": {
    "coding": [ {
      "system": "http://fhir.virtuallyhealthcare.co.uk/CodeSystem/tasks",
      "code": "WRITEBACK",
      "display": "Consultation Write Back"
    } ]
  },
  "focus": {
    "reference": "Encounter/3493d264-ab4e-4794-9622-555ed16b75aa",
    "identifier": {
      "system": "https://fhir.virtually.healthcare/Id/Encounter",
      "value": "d3aa0f25-6ceb-4baf-bb4f-8f53170c5231"
    }
  },
  "for": {
    "reference": "Patient/2cdce27e-9398-4367-8719-d872710e8e63"
  },
  "authoredOn": "2025-05-19T15:43:08+00:00",
  "owner": {
    "reference": "Organization/61a55cac-08d5-4cf8-a4d5-581f50962a82",
   

In [ ]:
# Due to way FHIRWorks is built we need to pause so that the update is processed

time.sleep(5)

### Get writeback tasks

In [202]:
import requests
import json

headersCDR = { "Accept": "application/fhir+json"}

cdrFHIRUrl = "http://localhost:8180/CDR/FHIR/R4"


parameters = {'_sort' : '-authored-on',
              'authored-on': 'gt2023-01-01',
              'status': 'accepted'}

tasks = []
response = requests.get(cdrFHIRUrl + '/Task',parameters,headers=headersCDR)
if response.status_code == 200:
    tasksJSON = json.loads(response.text)
    if 'entry' in tasksJSON:
        for entry in tasksJSON['entry']:
            if 'resource' in entry:
                print(entry['resource'])
                tasks.append(entry['resource'])


{'resourceType': 'Task', 'id': '8b12d352-b2dc-4647-a544-ea2ed30011e1', 'meta': {'versionId': '394', 'lastUpdated': '2025-06-26T12:12:57.090Z'}, 'identifier': [{'system': 'https://fhir.virtually.healthcare/Id/Task', 'value': '537fd315-8a60-4c02-a96e-473bd04a138d'}], 'status': 'accepted', 'intent': 'order', 'priority': 'routine', 'code': {'coding': [{'system': 'http://fhir.virtuallyhealthcare.co.uk/CodeSystem/tasks', 'code': 'WRITEBACK', 'display': 'Consultation Write Back'}]}, 'focus': {'reference': 'Encounter/3493d264-ab4e-4794-9622-555ed16b75aa', 'identifier': {'system': 'https://fhir.virtually.healthcare/Id/Encounter', 'value': 'd3aa0f25-6ceb-4baf-bb4f-8f53170c5231'}}, 'for': {'reference': 'Patient/2cdce27e-9398-4367-8719-d872710e8e63'}, 'authoredOn': '2025-05-19T15:43:08+00:00', 'owner': {'reference': 'Organization/61a55cac-08d5-4cf8-a4d5-581f50962a82', 'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code', 'value': 'F83004'}}, 'note': [{'time': '2025-05-19T15:44:0

### Get Consultation Note from CDR

Extract the full encounter from the CDR

In [203]:


CDRRecords = []
for task in tasks:
    encounter = task['focus']['identifier']
    parameters = {'identifier' : encounter['system'] + '|' + encounter['value']}

    responseCDR = requests.get(cdrFHIRUrl + '/Encounter/$extract-collection',parameters,headers=headersCDR)
    if responseCDR.status_code == 200:
        print("======= Response from extract collection ========")
        print(responseCDR.text)
        CDRRecords.append({
            "response": responseCDR.text,
            "task": task
        })

======= Response from extract collection ========
{
  "resourceType": "Bundle",
  "type": "collection",
  "entry": [ {
    "fullUrl": "https://ek1wj5eye3.execute-api.eu-west-2.amazonaws.com/dev/Encounter/3493d264-ab4e-4794-9622-555ed16b75aa",
    "resource": {
      "resourceType": "Encounter",
      "id": "3493d264-ab4e-4794-9622-555ed16b75aa",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2025-05-19T15:43:08.138Z"
      },
      "identifier": [ {
        "system": "https://fhir.virtually.healthcare/Id/Encounter",
        "value": "d3aa0f25-6ceb-4baf-bb4f-8f53170c5231"
      } ],
      "status": "finished",
      "class": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "AMB",
        "display": "ambulatory"
      },
      "type": [ {
        "text": "Virtually video"
      } ],
      "subject": {
        "reference": "Patient/2cdce27e-9398-4367-8719-d872710e8e63"
      },
      "participant": [ {
        "individual": {
     

### Convert to FHIR Message ADT_A04 (hard coded endpoint)



In [204]:
import uuid

FHIRMessages = []
for record in CDRRecords:
    resource = json.loads(record['response'])
    resource["type"] = "message"
    myuuid = uuid.uuid4()
    resource["identifier"] = {
        "system": "urn:ietf:rfc:3986",
        "value": 'urn:uuid:' + str(myuuid)
    }
    messageHeader = {
        "resourceType" : "MessageHeader",
        "eventCoding" : {
            "system" : "http://terminology.hl7.org/CodeSystem/v2-0003",
            "code" : "A04"
        },
        "destination" : [
            {
                "endpoint" : "http://ec2-18-130-139-120.eu-west-2.compute.amazonaws.com/emis",
                "receiver" : {
                    "identifier" : {
                        "system" : "https://fhir.nhs.uk/Id/ods-organization-code",
                        "value" : "F83004"
                    }
                }
            }
        ],
        "sender" : {
            "identifier" : {
                "system" : "https://fhir.nhs.uk/Id/ods-organization-code",
                "value" : "F83004"
            }
        },
        "source" : {
            "endpoint" : "http://ec2-18-130-139-120.eu-west-2.compute.amazonaws.com/emis"
        },
        "focus" : [
        ]
    }
    for entry in resource.get('entry', []):

       if 'resource' in entry:
           if 'resourceType' in entry['resource']:
               resourceType = entry['resource']['resourceType']
               if resourceType == 'Encounter':
                   print("Encounter")
                   messageHeader['focus'].append({
                       "reference" : entry['fullUrl'],
                       "type": "Encounter"
                   })
    resource["entry"].insert(0,{
        "fullUrl": "urn:uuid:" + str(myuuid),
        "resource": messageHeader
    })
    print(json.dumps(resource))
    FHIRMessages.append({
        "response" : json.dumps(resource),
        "task": record['task']
    })

Encounter
{"resourceType": "Bundle", "type": "message", "entry": [{"fullUrl": "urn:uuid:a518682e-8be4-4e01-a636-2037bb8f45c4", "resource": {"resourceType": "MessageHeader", "eventCoding": {"system": "http://terminology.hl7.org/CodeSystem/v2-0003", "code": "A04"}, "destination": [{"endpoint": "http://ec2-18-130-139-120.eu-west-2.compute.amazonaws.com/emis", "receiver": {"identifier": {"system": "https://fhir.nhs.uk/Id/ods-organization-code", "value": "F83004"}}}], "sender": {"identifier": {"system": "https://fhir.nhs.uk/Id/ods-organization-code", "value": "F83004"}}, "source": {"endpoint": "http://ec2-18-130-139-120.eu-west-2.compute.amazonaws.com/emis"}, "focus": [{"reference": "https://ek1wj5eye3.execute-api.eu-west-2.amazonaws.com/dev/Encounter/3493d264-ab4e-4794-9622-555ed16b75aa", "type": "Encounter"}]}}, {"fullUrl": "https://ek1wj5eye3.execute-api.eu-west-2.amazonaws.com/dev/Encounter/3493d264-ab4e-4794-9622-555ed16b75aa", "resource": {"resourceType": "Encounter", "id": "3493d264-

### Legacy QuestionnaireResponse Fix

New write will have more defined structure. Older writebacks need converting the to new structure.

In [205]:
import copy

def LegacyQuestionnaireResponseConversion(questionnaireResponse):
    questionnaireResponse['questionnaire'] = "https://fhir.virtually.healthcare/Questionnaire/ClinicalManagementPlan"
    newQR = copy.deepcopy(questionnaireResponse)
    newQR['item'] = [{
        "linkId": "LOINC/61149-1",
        "text": "Comments and advice",
        "item": []
    }]
    # missing linkId - assume it's the problem section
    problems = {
        "linkId": "LOINC/11450-4",
        "text": "Problem list",
        "item": []
    }
    problemsFound = False
    for item in questionnaireResponse.get('item', []):
        # exiting fat entries are plain questions and answers
        if 'answer' in item:
            newQR['item'][0]['item'].append({
                "linkId" : "questions",
                "item": [{
                        "linkId": "question",
                        "answer": [ {
                            "valueString": item['text']
                        }]
                    },
                    {
                        "linkId": "answer",
                        "answer": [ {
                            "valueString": item['answer'][0]["valueString"]
                        }]
                    }]
            })
        # problem management comes as a set of subitems.
        if 'item' in item:
            problem = {
                "linkId": "problem",
                "text": "Problem",
                "item": []
            }
            problemFound = False
            for problemitem in item.get('item', []):
                if 'linkId' not in problemitem:
                    newitem = {
                        'linkId' : 'problemCode',
                        'text' : "Problem Code",
                        'answer' : problemitem['answer']
                    }
                    problem['item'].append(newitem)
                if 'item' in problemitem:
                    for subitem in problemitem.get('item', []):
                        if subitem['linkId'] == 'problemStatus' or subitem['linkId'] == 'problemSignificance' or subitem['linkId'] == 'problemType' or subitem['linkId'] == 'problemExpectedDuration':
                            problem['item'].append(subitem)
                            problemFound = True
            if problemFound:
                problems['item'].append(problem)
                problemsFound = True

    if problemsFound:
        newQR['item'].append(problems)
    print(json.dumps(newQR))
    return newQR

for record in FHIRMessages:
    # begin fix the bundle
    resource = json.loads(record['response'])
    id = 0
    for entry in resource.get('entry', []):
        id += 1
        if 'resource' in entry:
            if 'resourceType' in entry['resource']:
                resourceType = entry['resource']['resourceType']
                if resourceType == 'QuestionnaireResponse':
                    # add in the questionnaire
                    if 'questionnaire' not in entry['resource']:
                        # legacy entries do not have a questionnaire
                        entry['resource'] = LegacyQuestionnaireResponseConversion(entry['resource'])
    record['response'] = json.dumps(resource)


{"resourceType": "QuestionnaireResponse", "id": "238c8ff7-f5d6-47be-a2ca-218c8732d68d", "meta": {"versionId": "1", "lastUpdated": "2025-05-19T15:43:08.943Z"}, "identifier": {"system": "https://fhir.virtually.healthcare/Id/QuestionnaireResponse", "value": "d3aa0f25-6ceb-4baf-bb4f-8f53170c5231"}, "status": "completed", "subject": {"reference": "Patient/2cdce27e-9398-4367-8719-d872710e8e63"}, "encounter": {"reference": "Encounter/3493d264-ab4e-4794-9622-555ed16b75aa", "identifier": {"system": "https://fhir.virtually.healthcare/Id/Encounter", "value": "d3aa0f25-6ceb-4baf-bb4f-8f53170c5231"}, "display": "Practitioner completed management plan"}, "authored": "2025-05-19T16:43:05+01:00", "author": {"reference": "PractitionerRole/8720bd06-6f81-454c-bf92-2be7f505a0e6"}, "item": [{"linkId": "LOINC/61149-1", "text": "Comments and advice", "item": [{"linkId": "questions", "item": [{"linkId": "question", "answer": [{"valueString": "Comments and advice"}]}, {"linkId": "answer", "answer": [{"valueStr

### Check Consultation does not already exist

TODO Check for duplicates. Should be a call to the Composition endpoint


In [206]:
emisFHIRUrl = "http://localhost:8180/EMIS/FHIR/R4"
headersEMIS = {"Accept": "application/fhir+json",
               "ODS_CODE": "F83004"}

for record in CDRRecords:
    resource = json.loads(record['response'])
    id = 0
    patientId = ''
    encounterId = ''
    for entry in resource.get('entry', []):
        id += 1
        if 'resource' in entry:
            if 'resourceType' in entry['resource']:
                resourceType = entry['resource']['resourceType']
                if resourceType == 'Patient':
                    if 'identifier' in entry['resource']:
                        for identifier in entry['resource'].get('identifier', []):
                            if 'system' in identifier and identifier['system'] == 'https://emis.com/Id/Patient/DBID':
                                patientId = identifier['value']
                    print(patientId)
                if resourceType == 'Encounter':
                    if 'identifier' in entry['resource']:
                        for identifier in entry['resource'].get('identifier', []):
                            if 'system' in identifier and identifier['system'] == 'https://fhir.virtually.healthcare/Id/Encounter':
                                encounterId = identifier['value']
                    print(encounterId)
    if (patientId != '' and encounterId != ''):
        url = emisFHIRUrl + f'/Composition?patient={patientId}'
        responseComposition = requests.get(url, headers=headersEMIS)
        print(responseComposition.text)
        resource = json.loads(responseComposition.text)
        for entry in resource.get('entry', []):
            if 'resourceType' in entry['resource'] and entry['resource']['resourceType'] == 'Composition':
                print('Have composition')
                if 'encounter' in entry['resource'] and 'identifier' in entry['resource']['encounter'] and entry['resource']['encounter']['identifier']['value'] == encounterId :
                    print('DUPLICATE SHOULD GRACEFULLY FINISH')


d3aa0f25-6ceb-4baf-bb4f-8f53170c5231
500837
{
  "resourceType": "Bundle",
  "id": "7af3ba4c-645a-4580-9d14-ffda96c80188",
  "meta": {
    "lastUpdated": "2025-06-26T12:13:00.729+00:00"
  },
  "type": "searchset",
  "total": 0,
  "link": [ {
    "relation": "self",
    "url": "http://localhost:8180/EMIS/FHIR/R4/Composition?patient=500837"
  } ]
}
f7ff9fd2-11f7-4a4e-9d84-dfccc669c7e9
500029
{
  "resourceType": "Bundle",
  "id": "69afce92-f660-4687-8310-ea8f7675ef67",
  "meta": {
    "lastUpdated": "2025-06-26T12:13:03.570+00:00"
  },
  "type": "searchset",
  "total": 127,
  "link": [ {
    "relation": "self",
    "url": "http://localhost:8180/EMIS/FHIR/R4/Composition?patient=500029"
  }, {
    "relation": "next",
    "url": "http://localhost:8180/EMIS/FHIR/R4?_getpages=11bb707d-e77f-4c43-aa69-782f783ba661&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "http://localhost:8180/EMIS/FHIR/R4/Composition/f7ff9fd2-11f7-4a4e-9d84-dfccc669c7e

### FHIR Validation (Testing)

TODO FHIR Validate - This takes time to run. So only provide a basic example.

`POST {baseUrl}/ESB/R4/$validate`

This checks the FHIR payload is valid against [Virtually Healthcare Implementation Guide](https://virtually-healthcare.github.io/R4/)


In [207]:
esbFHIRUrl = "http://localhost:8181/ESB/R4"
headersESB = {"Content-Type": "application/fhir+json",
               "ODS_CODE": "F83004"}

for record in FHIRMessages:
    # begin fix the bundle
    resource = json.loads(record['response'])

    #print(json.dumps(resource))
    responseValidate = requests.post(esbFHIRUrl + '/$validate', json.dumps(resource), headers=headersESB)
    print("======= Response from FHIR Validation ========")
    print("Status code = " + str(responseValidate.status_code))
    operationOutcome = json.loads(responseValidate.text)
    failed = False

    if 'issue' in operationOutcome:
        print("======= Error ========")
        for issue in operationOutcome['issue']:
            if issue['severity'] == 'error':
                ignore = False
                if 'details' in issue:
                    if '307321000000107' in issue['details']['text']:
                        ignore = True
                if not ignore:
                    failed = True
                    if 'details' in issue:
                        print("Issue: " + issue['details']['text'] + ' [' + issue['severity'] + ']')
                    if 'diagnostics' in issue:
                        print("Issue: " + issue['diagnostics'] + ' [' + issue['severity'] + ']')
                    if 'expression' in issue:
                        print(issue['expression'])
                    print('')
        print("======= Warning ========")
        for issue in operationOutcome['issue']:
            if issue['severity'] == 'warning':
                if 'details' in issue:
                    print("Issue: " + issue['details']['text'] + ' [' + issue['severity'] + ']')
                if 'diagnostics' in issue:
                    print("Issue: " + issue['diagnostics'] + ' [' + issue['severity'] + ']')
                if 'expression' in issue:
                    print(issue['expression'])
                print('')

    if failed:
        print("FAILED Validation")
         #record['task']['status'] = 'failed'
         #response = requests.put(cdrFHIRUrl + '/Task/' + task['id'],json.dumps(task),headers=headersCDR)
        #print(response.text)
        #break


======= Response from FHIR Validation ========
Status code = 200
======= Error ========
======= Warning ========
Issue: None of the codings provided are in the value set 'IdentifierType' (http://hl7.org/fhir/ValueSet/identifier-type|4.0.1), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://terminology.hl7.org/CodeSystem/v2-0203#NH) [warning]
['Bundle.entry[2].resource/*Patient/2cdce27e-9398-4367-8719-d872710e8e63*/.identifier[1].type']

Issue: None of the codings provided are in the value set 'ServiceDeliveryLocationRoleType' (http://terminology.hl7.org/ValueSet/v3-ServiceDeliveryLocationRoleType|3.0.0), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#307321000000107) [warning]
['Bundle.entry[3].resource/*Location/daabb4e8-dab2-4237-a957-122fe83e62a3*/.type[0]']

Issue: Value

### Convert to EMISOpen

In [208]:
emisFHIRUrl = "http://localhost:8180/EMIS/FHIR/R4"
headersEMIS = {"Content-Type": "application/fhir+json",
               "ODS_CODE": "F83004"}

EMISOpenRecords = []
for record in FHIRMessages:
    responseEMISTransform = requests.post(emisFHIRUrl + '/Bundle/$transform-EMISOpen', record['response'], headers=headersEMIS )
    print("======= Response from transform to EMIS Open ========")
    print(responseEMISTransform.text)
    EMISOpenRecords.append({
        "response" : responseEMISTransform.text,
        "task": record['task']
    })


======= Response from transform to EMIS Open ========
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<MedicalRecord PatientID="500837" xmlns="http://www.e-mis.com/emisopen/MedicalRecord">
    <ConsultationList>
        <Consultation>
            <GUID>d3aa0f25-6ceb-4baf-bb4f-8f53170c5231</GUID>
            <AssignedDate>20/05/2025 04:00:00</AssignedDate>
            <UserID>
                <RefID>2</RefID>
            </UserID>
            <LocationTypeID>
                <RefID>1</RefID>
            </LocationTypeID>
            <OriginalAuthor>
                <User>
                    <RefID>2</RefID>
                </User>
            </OriginalAuthor>
            <ElementList>
                <ConsultationElement>
                    <RefID>1</RefID>
                    <Header>
                        <Term>Comment</Term>
                    </Header>
                    <Event>
                        <RefID>1</RefID>
                        <GUID>90695ae5-2945-4fce-

### Send Consultation Note

Send to EMISOpen formated interactions to EMIS IM1 Transaction API.

In [211]:
sendResponses = []
for EMISOpen in EMISOpenRecords:

    headersEMIS = {"Content-Type": "application/fhir+json",
               "ODS_CODE": "F83004"}
    body = {
        "resourceType": "Parameters",
        "parameter" : [ {
            "name" : "EMISOpen",
            "valueString": EMISOpen['response']  }]
    }

    responseEMISSend = requests.post(emisFHIRUrl + '/$send-EMISOpen', json.dumps(body), headers=headersEMIS )
    print("======= Send to EMIS Open ========")
    print(responseEMISSend.text)
    sendResponses.append({
        "response" : responseEMISSend.text,
        "task": EMISOpen['task']
    }
)

======= Send to EMIS Open ========
{
  "resourceType": "OperationOutcome",
  "issue": [ {
    "severity": "error",
    "code": "exception",
    "diagnostics": "HTTP operation failed invoking http://172.31.22.225/emis/MedicalRecord/500837? with statusCode: 400"
  } ]
}
======= Send to EMIS Open ========
{
  "resourceType": "OperationOutcome",
  "issue": [ {
    "severity": "information",
    "code": "informational"
  } ]
}


### Update Send Consulation Note workflow status

Update Task to say it is completed or failed

In [210]:
headersCDR = {"Content-Type": "application/fhir+json", "Accept": "application/fhir+json"}

for sendResponse in sendResponses:
    task = sendResponse['task']
    task['status'] = 'completed'
    response = requests.put(cdrFHIRUrl + '/Task/' + task['id'],json.dumps(task),headers=headersCDR)
    print(response.text)

{
  "resourceType": "Task",
  "id": "8b12d352-b2dc-4647-a544-ea2ed30011e1",
  "meta": {
    "versionId": "396",
    "lastUpdated": "2025-06-26T12:14:58.330Z"
  },
  "identifier": [ {
    "system": "https://fhir.virtually.healthcare/Id/Task",
    "value": "537fd315-8a60-4c02-a96e-473bd04a138d"
  } ],
  "status": "completed",
  "intent": "order",
  "priority": "routine",
  "code": {
    "coding": [ {
      "system": "http://fhir.virtuallyhealthcare.co.uk/CodeSystem/tasks",
      "code": "WRITEBACK",
      "display": "Consultation Write Back"
    } ]
  },
  "focus": {
    "reference": "Encounter/3493d264-ab4e-4794-9622-555ed16b75aa",
    "identifier": {
      "system": "https://fhir.virtually.healthcare/Id/Encounter",
      "value": "d3aa0f25-6ceb-4baf-bb4f-8f53170c5231"
    }
  },
  "for": {
    "reference": "Patient/2cdce27e-9398-4367-8719-d872710e8e63"
  },
  "authoredOn": "2025-05-19T15:43:08+00:00",
  "owner": {
    "reference": "Organization/61a55cac-08d5-4cf8-a4d5-581f50962a82",
  